# Hospital Waste Management LCA

In [2]:
import pandas as pd
import olca_ipc as olca
import uuid
import math
from datetime import datetime
import numpy as np
import sys

# from matplotlib import pyplot as plt
# import matplotlib.mlab as mlab
# from matplotlib import rcParams
# import matplotlib.patches as mpatches
# import seaborn as sns

## Preliminary assumptions

For this, we choose to ignore the impacts of transporting waste from the hospital to the various disposal facilities. This is because we are focusing on large hospitals that are located in cities with these facilities easily accessible. Because of the locality of these facilities, we ignore the social, econonmic, and environmental impacts of the transportation.

## Waste Stream Composition Scenarios

In [3]:
# types of waste
# Columns (must sum to 1.0):
# 0 - Hard plastic
# 1 - Soft plastic
# 2 - Paper
# 3 - Food
# 4 - Textiles
# 5 - Glass
# 6 - Metal
# 7 - Electronic waste
# 8 - Sharps

def normalize(arr):
    return arr / arr.sum()

# Baseline Input Scenario 0 (From Hsu et al., 2020)
waste_compsition_0 = np.array([0.195, 0.451, 0.184, 0.062, 0.035, 0.012, 0.004, 0.001, 0.036])
waste_compsition_0 = normalize(waste_compsition_0)

# Data from Aarushi will go here

# Input Scenario 1
waste_compsition_1 = np.array([0.4, 0.2, 0.2, 0.1, 0.02, 0.08])

# Input Scenario 2
waste_compsition_2 = np.array([0.4, 0.2, 0.2, 0.1, 0.02, 0.08])

## Hazardous Waste Proportion Minimum

In [4]:
# types of waste
# Columns (must sum to 1.0):
# 0 - Hard plastic
# 1 - Soft plastic
# 2 - Paper
# 3 - Food
# 4 - Textiles
# 5 - Glass
# 6 - Metal
# 7 - Electronic waste
# 8 - Sharps

# Baseline all waste is treated as hazardous
hazard_prop_0 = np.array([1.0] * 9)
print(hazard_prop_0)

[1. 1. 1. 1. 1. 1. 1. 1. 1.]


## Waste Disposal Method Scenarios

In [30]:
# Waste Disposal Methods
# Rows:
# 0 - Proportion to Landfill
# 1 - Proportion to Recycling
# 2 - Proportion to WtE
# 3 - Proportion to Composting
# 4 - Proportion to Incinerator
# Columns (must sum to 1.0):
# 0 - Hard plastic
# 1 - Soft plastic
# 2 - Paper
# 3 - Food
# 4 - Textiles
# 5 - Glass
# 6 - Metal
# 7 - Electronic waste
# 8 - Sharps

# Data must be decided on by the team

# Worst-case method 0 (all landfill, hazardous waste to incinerators)
waste_treatment_0 = np.array([[1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]])

# Waste treatment method 1
waste_treatment_1 = np.array([[1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]])
# Waste treatment method 2
waste_treatment_2 = np.array([[1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00],
                              [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]])

## Impact Factor Variables

In [31]:
# Emissions Factors
# Rows:
# 0 - Landfill
# 1 - Recycling
# 2 - WtE (incineration)
# 3 - Composting
# 4 - Incinerator
# 5 - Sanitize and Reuse
# Columns:
# 0 - Hard plastic (HDPE)
# 1 - Soft plastic (LDPE)
# 2 - Paper
# 3 - Food
# 4 - Textiles
# 5 - Glass
# 6 - Metal
# 7 - Electronic waste
# 8 - Sharps

# Data must be found for 'impact_cost' and 'impact_co2_eq'. 'impact_community' can be decided upon subjectively

inf = sys.float_info.max

# Social cost of carbon (USD/ton CO2-eq)
social_cost_carbon = 190.0

# Cost of process (values in $/short ton waste)
impact_cost = np.array([[73.45, 73.45, 73.45, 73.45, 73.45, 73.45, 73.45, 73.45, 73.45],
                        [106.0, 106.0, 106.0, 106.0, 106.0, 106.0, 106.0, 106.0, 106.0],
                        [63.69, 63.69, 63.69, 63.69, 63.69, 63.69, 63.69, 63.69, 63.69],
                        [67.53, 67.53, 67.53, 67.53, 67.53, 67.53, 67.53, 67.53, 67.53],
                        [63.69, 63.69, 63.69, 63.69, 63.69, 63.69, 63.69, 63.69, 63.69],
                        [997.9, 997.9, 997.9, 997.9, 997.9, 997.9, 997.9, 997.9, 997.9]])

# CO2 Emissions of process (ton CO2-eq/short ton waste)
impact_co2_eq = np.array([[0.02, 0.02, 0.80, 0.58, 0.02, 0.02, 0.02, 0.02, 0.02],
                          [0.21,  inf, 0.07,  inf,  inf, 0.05, 0.23,  inf, 0.32],
                          [2.80, 2.80, 0.05, 0.05, 1.68, 0.01, 0.01,  inf, 0.01],
                          [ inf,  inf,  inf, 0.15,  inf,  inf,  inf,  inf,  inf],
                          [2.80, 2.80, 0.05, 0.05, 1.68, 0.01, 0.01,  inf, 0.01],
# TODO: need data for the emissions profile from sanitization of materials
                          [0.02, 0.02, 0.80, 0.58, 0.02, 0.02, 0.02, 0.02, 0.02]])

# TODO: Need to get values for this
# Waste mass  (ton CO2-eq/short ton waste)
impact_mass   = np.array([[0.02, 0.02, 0.80, 0.58, 0.02, 0.02, 0.02, 0.02, 0.02],
                          [0.21,  inf, 0.07,  inf,  inf, 0.05, 0.23,  inf, 0.32],
                          [2.80, 2.80, 0.05, 0.05, 1.68, 0.01, 0.01,  inf, 0.01],
                          [ inf,  inf,  inf, 0.15,  inf,  inf,  inf,  inf,  inf],
                          [2.80, 2.80, 0.05, 0.05, 1.68, 0.01, 0.01,  inf, 0.01],
                          [0.02, 0.02, 0.80, 0.58, 0.02, 0.02, 0.02, 0.02, 0.02]])

# TODO: Need to get values for this
# Energy impact  (MW/short ton waste)
impact_energy   = np.array([[0.02, 0.02, 0.80, 0.58, 0.02, 0.02, 0.02, 0.02, 0.02],
                            [0.21,  inf, 0.07,  inf,  inf, 0.05, 0.23,  inf, 0.32],
                            [2.80, 2.80, 0.05, 0.05, 1.68, 0.01, 0.01,  inf, 0.01],
                            [ inf,  inf,  inf, 0.15,  inf,  inf,  inf,  inf,  inf],
                            [2.80, 2.80, 0.05, 0.05, 1.68, 0.01, 0.01,  inf, 0.01],
                            [0.02, 0.02, 0.80, 0.58, 0.02, 0.02, 0.02, 0.02, 0.02]])

# dry digestate with curing []
# wet digestate with curing []

# Community impact of process
impact_total  = social_cost_carbon * impact_co2_eq + impact_cost

C:\Users\Astro\AppData\Local\Temp\ipykernel_18736\722741163.py:57: RuntimeWarning: overflow encountered in multiply
  impact_total  = social_cost_carbon * impact_co2_eq + impact_cost


Municipal Solid Waste (MSW) Landfill	$60.34
MSW Recycling Center	$52.33
MSW Waste-to-Energy Facility	$102.13
Composting Facility	$20-$80
Incinerator Facility	$100-$200

Data sources:

Tipping fees:
* Tipping fee (landfill, PA, 2020): https://assets.website-files.com/602b605a94315c491a4fb4b8/610aed49f0ef85f4566dbe3f_Tipping-Fees-Data-Summary%20Jan%202021.pdf
* Tipping fee (recycling, PA): https://www.phila.gov/media/20210614135759/Municipal-Waste-Management-Plan-appendix-C.pdf
* WtE Facility, PA: https://www.covanta.com/where-we-are/our-facilities/plymouth
* Composting (to energy, PA): https://www.phila.gov/media/20210614135413/Municipal-Waste-Management-Plan-202010.pdf
* https://kleinmanenergy.upenn.edu/research/publications/powering-our-future-with-trash/ 
* Autoclave cost: https://www.sciencedirect.com/science/article/pii/S0956053X12004606

Emissions data:
* Plastic types (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7151894/)
* GHG 2022 Emission factors (https://www.epa.gov/climateleadership/ghg-emission-factors-hub)
* 
* 

Social Cost of Carbon:
* https://www.brookings.edu/articles/what-is-the-social-cost-of-carbon/#:~:text=The%20social%20cost%20of%20carbon%20(SCC)%20is%20an%20estimate%20of,a%20ton%20of%20carbon%20emissions.
* https://www.epa.gov/system/files/documents/2022-11/epa_scghg_report_draft_0.pdf

In [32]:
disposal_method_names = ["Landfill", "Recycling", "Waste-to-Energy", "Composting", "Incineration", "Reuse"]
composition_type_names = ["Hard plastic", "Soft plastic", "Paper", "Food", "Textiles", "Glass", "Metal", "Electronic waste", "Sharps"]

In [33]:
def assess_impact(composition, methods, factors, name=None):
    print(composition.shape, methods.shape, factors.shape)
    final_impact_matrix = (composition * methods) * factors
    verbose = name != None
    if verbose:
        print("-" * 50)
        print("Analyzing Scenario: " + str(name) + '\n------------------------\n')
        print(final_impact_matrix)
        print("\nTotal " + str(name) + ": " + str(sum(sum(final_impact_matrix)).round(3)) + " per ton waste")
        print("\n\tBreakdown per ton:\n\t------------------------------")
        for i in range(len(disposal_method_names)):
            print('\t' + (disposal_method_names[i] + " " + name).ljust(24) + str(sum(final_impact_matrix[i]).round(3)))
        print()
        for i in range(len(composition_type_names)):
            print('\t' + (composition_type_names[i] + " " + name).ljust(24) + str(sum(final_impact_matrix)[i].round(3)))
        print()
        print("-" * 50)
    return final_impact_matrix

In [34]:
cost_assessment = assess_impact(waste_compsition_0, waste_treatment_0, impact_cost, name='Cost')

(9,) (6, 9) (6, 9)
--------------------------------------------------
Analyzing Scenario: Cost
------------------------

[[14.61505102 33.8019898  13.79061224  4.64683673  2.62321429  0.89938776
   0.29979592  0.07494898  2.69816327]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]]

Total Cost: 73.45 per ton waste

	Breakdown per ton:
	------------------------------
	Landfill Cost           73.45
	Recycling Cost          0.0
	Waste-to-Energy Cost    0.0
	Composting Cost         0.0
	Incineration Cost       0.0
	Reuse 

In [35]:
co2_assessment = assess_impact(waste_compsition_0, waste_treatment_0, impact_co2_eq, name='CO2')

(9,) (6, 9) (6, 9)
--------------------------------------------------
Analyzing Scenario: CO2
------------------------

[[3.97959184e-03 9.20408163e-03 1.50204082e-01 3.66938776e-02
  7.14285714e-04 2.44897959e-04 8.16326531e-05 2.04081633e-05
  7.34693878e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]]

Total CO2: 0.202 per ton w

In [36]:
co2_assessment = assess_impact(waste_compsition_0, waste_treatment_0, impact_total, name='Total Cost')

(9,) (6, 9) (6, 9)
--------------------------------------------------
Analyzing Scenario: Total Cost
------------------------

[[15.37117347 35.55076531 42.32938776 11.61867347  2.75892857  0.94591837
   0.31530612  0.07882653  2.8377551 ]
 [ 0.                 nan  0.                 nan         nan  0.
   0.                 nan  0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.                 nan  0.        ]
 [        nan         nan         nan  0.                 nan         nan
          nan         nan         nan]
 [ 0.          0.          0.          0.          0.          0.
   0.                 nan  0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.        ]]

Total Total Cost: nan per ton waste

	Breakdown per ton:
	------------------------------
	Landfill Total Cost     111.807
	Recycling Total Cost    nan
	Waste-to-Energy Total Costnan
	Composting Total Cost   nan
	Incineration 

C:\Users\Astro\AppData\Local\Temp\ipykernel_18736\3382307009.py:3: RuntimeWarning: invalid value encountered in multiply
  final_impact_matrix = (composition * methods) * factors
